### 좋아요수 / 댓글 분산 적용 기반 추천(명곡?)
- Input(title, artist) : contains로 찾기 => 결과 songId 한 개만 반환하도록 한다.
- I am 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 -> 가사분석해서 비슷한 노래 찾기 top5
- I am 찾기 -> 가사분석해서 비슷한 노래 찾기 -> 전체 (좋아요+댓글) -> 3번째 분산 데이터들에서 top5
- 반환된 songId 들의 정보 출력

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/melon_song.csv')
df.head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201.0,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138267,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859
1,35609035,인생은 뷰티풀,김호중,발라드 국내영화,인생은 뷰티풀: 비타돌체,김호중,김호중,20220907.0,https://cdnimg.melon.co.kr/cm2/album/images/11...,197,17322,하나만 알고 둘은\n잘 모르는 사람입니다\n하루는 비틀거리고\n하루는 뒤뚱거리고\n...,457536944 474137187


In [3]:
# 데이터 처리를 위한 작업
df.date.fillna(0, inplace=True)
df['date'] = df.date.astype(int).astype(str)
df.fillna('', inplace=True)
df['comment_like_total'] = df.comment + df.like
df['songId'] = df.songId.astype(str)

In [4]:
# 한글 불용어 처리
with open('data/불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

In [5]:
from konlpy.tag import Okt
okt = Okt()

In [6]:
lyrics = []
for lyric in df.lyric:
    lyric = lyric.replace('\n', ' ')
    morphs = okt.morphs(lyric, stem=True)
    tmp = [word for word in morphs if word not in stop_words]
    lyrics.append(' '.join(tmp))
df['morphs'] = lyrics

In [7]:
# 학습할 데이터열 생성
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 3

In [8]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)

In [10]:
indices = pd.Series(df.index, index=df.songId)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [12]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(31).tail(30).index
    return df.songId.iloc[song_indices]

In [13]:
a = get_recommendation('32720013', cosine_sim).to_frame()
a.head()

,songId
4144,35595137
3822,36034936
6100,30500281
3118,3815739
3511,35546497


In [14]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
sim_scores = pd.Series(cosine_sim_total[indices['32720013']])
sim_scores.sort_values(ascending=False).head(11).tail(10)

4144    0.595280
3822    0.449026
6100    0.328255
3118    0.320006
3511    0.316327
4040    0.290627
3816    0.287135
4414    0.283626
3503    0.277437
4773    0.260115
dtype: float64

In [15]:
# 찾고 싶은 구간 정하기
numbers = df['comment_like_total']
sorted_numbers = np.sort(numbers)
q1 = np.percentile(sorted_numbers, 25)
q2 = np.percentile(sorted_numbers, 50)
filtered_data = df[(df['comment_like_total'] >= q1) & (df['comment_like_total'] < q2)]
filtered_data = filtered_data[['songId', 'comment_like_total']]
filtered_data.songId.values

array(['35837631', '35866717', '35879546', ..., '3080890', '2981597',
       '34183979'], dtype=object)

In [16]:
# 유사도 top 30 중 원하는 구간에 있는 songId 추출(유사도순)
d = a[a['songId'].isin(filtered_data.songId.values)].head(5)
d

,songId
6100,30500281
4040,35846577
4414,36342068
4779,36327988
4002,36471556


In [17]:
filtered_df = df[df['songId'].isin(a.songId.values[:5])].to_dict('records')
filtered_df

[{'songId': '3815739',
  'title': 'Like this',
  'artist': '원더걸스',
  'genre': '댄스',
  'album': 'Wonder Party',
  'lyricist': '박진영',
  'composer': '박진영',
  'date': '20120603',
  'img': 'https://cdnimg.melon.co.kr/cm/album/images/021/25/515/2125515_500.jpg/melon/resize/282/quality/80/optimize',
  'comment': 55,
  'like': 30075,
  'lyric': 'Like this yo Like this \n위 아래로 흔들어 Like this\nLike this yo Like this \n몸을 돌려 좌우로 Like this \nLike this yo Like this \n파도처럼 내려가 Like this \nLike this yo Like this \n다같이 Stop 자 흔들어봐 \n나른하고 지치고 따분한\n지친 일상을 벗어나 \n뭔가 상큼하고 새롭고 신나는 \n일을 찾고 있다면 \n나를 어서 따라와 \n상쾌한 바람이 네게 들어와 \n너를 깨어나게 해줄게 \n여기 내 손을 잡아 오우 \nCome on baby come on\n귀를 열어봐 \nFeel it baby feel it\n어서 느껴봐 \nGo on baby go on\n용기를 내봐 \n1 2 3 자 Ready set go \nCome on baby come on\n귀를 열어봐 \nFeel it baby feel it\n어서 느껴봐 \nGo on baby go on\n용기를 내봐 \n1 2 3 자 Ready set go \nLike this yo Like this \n위 아래로 흔들어 Like this\nLike this yo Like this \n몸을 돌려 좌우로 Like this \nLike this yo Like this \n파도처럼 내려가 Like this \